In [13]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import numpy as np
import wandb
import torch.nn.functional as F
import torch.optim as optim

# 요구사항 1

In [14]:
# WANDB 프로젝트 초기화
# wandb.init을 사용하여 "titanic-pytorch"라는 이름으로 Weights & Biases(WANDB) 프로젝트를 초기화함.
# 이를 통해 실험 결과를 추적하고 시각화할 수 있음.
wandb.init(project="titanic-pytorch")

# TitanicDataset 정의
# PyTorch의 Dataset 클래스를 상속하여 TitanicDataset을 정의함.
# X는 입력 데이터(특징), y는 레이블(목표 값)임.

class TitanicDataset(Dataset):
    def __init__(self, X, y):
        # 입력 데이터 X를 FloatTensor로 변환하여 self.X에 저장.
        # y(레이블)를 LongTensor로 변환하여 self.y에 저장.
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        # 데이터셋의 크기를 반환하는 메서드.
        # len(self.X)를 호출하여 데이터의 총 개수를 반환함.
        return len(self.X)

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터를 가져옴.
        # feature는 self.X에서 입력 데이터를, target은 self.y에서 레이블을 가져옴.
        # 두 값을 딕셔너리 형태로 반환하여 모델 학습에 사용될 수 있도록 함.
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}


In [15]:
# TitanicTestDataset 정의
# PyTorch의 Dataset 클래스를 상속하여 TitanicTestDataset을 정의함.
# 테스트 데이터셋으로, 레이블(y)이 없고 입력 데이터(X)만 처리함.

class TitanicTestDataset(Dataset):
    def __init__(self, X):
        # 입력 데이터 X를 FloatTensor로 변환하여 self.X에 저장.
        self.X = torch.FloatTensor(X)

    def __len__(self):
        # 데이터셋의 크기를 반환하는 메서드.
        # len(self.X)를 호출하여 데이터의 총 개수를 반환함.
        return len(self.X)

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 데이터를 가져옴.
        # feature는 self.X에서 입력 데이터를 가져와 반환함.
        # 테스트 데이터이므로 타겟(레이블)이 없으며, 'input'만 딕셔너리 형태로 반환함.
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        # 데이터셋의 정보(크기 및 입력 데이터의 모양)를 문자열로 반환하는 메서드.
        # len(self.X)는 데이터셋 크기, self.X.shape는 입력 데이터의 형태를 나타냄.
        return "Data Size: {0}, Input Shape: {1}".format(
            len(self.X), self.X.shape
        )


In [16]:
# 데이터 전처리 함수들 정의

# 결측된 나이(Age) 값을 평균 나이로 채움.
def get_preprocessed_dataset_1(all_df):
    all_df['Age'] = all_df['Age'].fillna(all_df['Age'].mean())
    return all_df

# 성별(Sex)을 숫자로 변환. 남성은 0, 여성은 1로 맵핑.
def get_preprocessed_dataset_2(all_df):
    all_df['Sex'] = all_df['Sex'].map({'male': 0, 'female': 1})
    return all_df

# 'Cabin', 'Ticket', 'Name' 열 삭제. 모델 학습에 불필요한 데이터를 제거.
def get_preprocessed_dataset_3(all_df):
    all_df = all_df.drop(columns=['Cabin', 'Ticket', 'Name'])
    return all_df

# 탑승 항구(Embarked) 결측값을 최빈값으로 채우고, 'C', 'Q', 'S'를 각각 0, 1, 2로 맵핑.
def get_preprocessed_dataset_4(all_df):
    all_df['Embarked'] = all_df['Embarked'].fillna(all_df['Embarked'].mode()[0])
    all_df['Embarked'] = all_df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
    return all_df

# 운임(Fare)에 로그 변환을 적용하여 데이터의 분포를 정규화함.
def get_preprocessed_dataset_5(all_df):
    all_df['Fare'] = np.log1p(all_df['Fare'])
    return all_df

# 형제 자매(SibSp)와 부모 자식(Parch)을 합하여 가족 크기(FamilySize) 열을 생성.
# 만약 'SibSp' 또는 'Parch' 열이 없으면 KeyError를 발생시킴.
def get_preprocessed_dataset_6(all_df):
    if 'SibSp' in all_df.columns and 'Parch' in all_df.columns:
        all_df['FamilySize'] = all_df['SibSp'] + all_df['Parch']  # FamilySize 생성
        print("FamilySize 열 생성 완료!")  # 확인용 출력
    else:
        raise KeyError("'SibSp' 또는 'Parch' 열이 없습니다.")
    return all_df


In [17]:
# 데이터셋을 전처리하는 함수 정의
# Titanic 데이터셋을 전처리하고, 학습, 검증, 테스트 데이터셋을 반환함.

def get_preprocessed_dataset():
    # 현재 작업 중인 디렉토리 경로를 가져옴.
    CURRENT_FILE_PATH = os.getcwd()

    # train.csv와 test.csv 파일 경로를 생성함.
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # train.csv와 test.csv 파일을 읽어 각각 데이터프레임으로 저장함.
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    # train_df와 test_df를 하나의 데이터프레임으로 결합함. Survived 열을 기준으로 결합됨.
    all_df = pd.concat([train_df, test_df], sort=False)

    # 각각의 전처리 함수들을 호출하여 데이터프레임을 전처리함.
    all_df = get_preprocessed_dataset_1(all_df)
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)

    # Survived 값이 존재하는 행은 train_df로, 존재하지 않는 행은 test_df로 나눔.
    train_df = all_df.loc[all_df['Survived'].notnull()]
    test_df = all_df.loc[all_df['Survived'].isnull()]

    # 사용할 특징(feature)들과 라벨(label)을 설정함.
    features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize']
    label = 'Survived'

    # 학습 데이터셋(X_train)과 라벨(y_train)을 배열로 변환.
    X_train = train_df[features].values
    y_train = train_df[label].values

    # 테스트 데이터셋(X_test)을 배열로 변환.
    X_test = test_df[features].values

    # TitanicDataset을 사용해 전체 학습 데이터셋과 테스트 데이터셋을 생성함.
    full_train_dataset = TitanicDataset(X_train, y_train)
    test_dataset = TitanicTestDataset(X_test)

    # 학습 데이터셋의 80%는 훈련(train) 데이터로, 20%는 검증(validation) 데이터로 나눔.
    train_size = int(0.8 * len(full_train_dataset))
    val_size = len(full_train_dataset) - train_size

    # random_split을 사용하여 학습 데이터셋과 검증 데이터셋을 나눔.
    train_dataset, validation_dataset = random_split(full_train_dataset, [train_size, val_size])

    # 전처리된 학습, 검증, 테스트 데이터셋을 반환함.
    return train_dataset, validation_dataset, test_dataset


In [37]:
# 모델 정의
# PyTorch의 nn.Module을 상속받아 MyModel 클래스를 정의함.
# 이 모델은 간단한 완전 연결 신경망(fully connected neural network)으로 구성됨.

class MyModel_relu(nn.Module):
    def __init__(self, n_input, n_output):
        # 부모 클래스(nn.Module)의 초기화 메서드를 호출하여 초기화.
        super(MyModel_relu, self).__init__()
        
        # nn.Sequential을 사용해 모델의 레이어들을 차례대로 정의.
        # 입력층: n_input -> 30 유닛 (Linear 레이어)
        # 활성화 함수로 ReLU를 사용함.
        # 은닉층: 30 -> 30 유닛 (Linear 레이어) + ReLU 활성화 함수
        # 출력층: 30 -> n_output 유닛 (Linear 레이어)
        self.model = nn.Sequential(
            nn.Linear(n_input, 30),  # 입력층: n_input -> 30
            nn.ReLU(),  # 활성화 함수 ReLU
            nn.Linear(30, 30),  # 은닉층: 30 -> 30
            nn.ReLU(),  # 활성화 함수 ReLU
            nn.Linear(30, n_output)  # 출력층: 30 -> n_output
        )

    # forward 메서드는 순전파(입력에서 출력으로) 과정을 정의.
    # 입력 데이터 x를 모델에 전달하고 결과값을 반환.
    def forward(self, x):
        x = self.model(x)
        return x


In [38]:
# 모델 정의
# PyTorch의 nn.Module을 상속받아 MyModel 클래스를 정의함.
# 이 모델은 간단한 완전 연결 신경망(fully connected neural network)으로 구성됨.

class MyModel_sigmoid(nn.Module):
    def __init__(self, n_input, n_output):
        # 부모 클래스(nn.Module)의 초기화 메서드를 호출하여 초기화.
        super(MyModel_sigmoid, self).__init__()
        
        # nn.Sequential을 사용해 모델의 레이어들을 차례대로 정의.
        # 입력층: n_input -> 30 유닛 (Linear 레이어)
        # 활성화 함수로 Sigmoid를 사용함.
        # 은닉층: 30 -> 30 유닛 (Linear 레이어) + Sigmoid 활성화 함수
        # 출력층: 30 -> n_output 유닛 (Linear 레이어)
        self.model = nn.Sequential(
            nn.Linear(n_input, 30),  # 입력층: n_input -> 30
            nn.Sigmoid(),  # 활성화 함수 Sigmoid
            nn.Linear(30, 30),  # 은닉층: 30 -> 30
            nn.Sigmoid(),  # 활성화 함수 Sigmoid
            nn.Linear(30, n_output)  # 출력층: 30 -> n_output
        )

    # forward 메서드는 순전파(입력에서 출력으로) 과정을 정의.
    # 입력 데이터 x를 모델에 전달하고 결과값을 반환.
    def forward(self, x):
        x = self.model(x)
        return x


In [63]:
# 모델 학습 함수 정의
# 주어진 모델을 학습시키고 검증하는 함수 train_model을 정의함.
# 이 함수는 학습 손실과 검증 손실을 계산하고, WANDB에 로그를 기록함.

def train_model(model, train_loader, validation_loader, epochs=10):
    # 손실 함수로 CrossEntropyLoss 사용 (분류 문제에서 주로 사용됨).
    # criterion = nn.CrossEntropyLoss()
    criterion = nn.MultiMarginLoss()
    
    # Adam 옵티마이저를 사용하여 학습률 0.001로 설정.
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 에포크(epochs) 동안 학습을 반복함.
    for epoch in range(epochs):
        model.train()  # 모델을 학습 모드로 전환.
        train_loss = 0  # 한 에포크 동안의 훈련 손실을 저장할 변수.

        # 훈련 데이터로 학습.
        for batch in train_loader:
            inputs = batch['input']  # 입력 데이터를 가져옴.
            targets = batch['target']  # 타겟(레이블) 데이터를 가져옴.

            optimizer.zero_grad()  # 옵티마이저의 그라디언트 초기화.
            outputs = model(inputs)  # 모델을 통해 예측값을 얻음.
            loss = criterion(outputs, targets)  # 손실을 계산.
            loss.backward()  # 손실에 대한 그라디언트를 계산 (역전파).
            optimizer.step()  # 옵티마이저를 통해 모델 파라미터를 업데이트.

            train_loss += loss.item()  # 배치 손실을 총 학습 손실에 더함.

        model.eval()  # 모델을 평가 모드로 전환.
        validation_loss = 0  # 한 에포크 동안의 검증 손실을 저장할 변수.
        
        # 검증 데이터로 모델 평가. 평가 중에는 그라디언트 계산을 하지 않음.
        with torch.no_grad():
            for batch in validation_loader:
                inputs = batch['input']  # 입력 데이터를 가져옴.
                targets = batch['target']  # 타겟(레이블) 데이터를 가져옴.

                outputs = model(inputs)  # 모델을 통해 예측값을 얻음.
                loss = criterion(outputs, targets)  # 손실을 계산.
                validation_loss += loss.item()  # 배치 손실을 총 검증 손실에 더함.

        # WANDB에 학습 및 검증 손실을 로그로 기록.
        # wandb.log({
        #     'train_loss': train_loss / len(train_loader),  # 평균 학습 손실
        #     'validation_loss': validation_loss / len(validation_loader),  # 평균 검증 손실
        #     'epoch': epoch + 1  # 현재 에포크
        # })

        # 현재 에포크에 대한 학습 및 검증 손실을 출력.
        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader)}, Validation Loss: {validation_loss / len(validation_loader)}")


# 요구사항 2, 3

In [64]:
if __name__ == "__main__":
    # WANDB 초기화
    # Weights & Biases(WANDB)에서 실험 추적을 위해 프로젝트 "titanic-pytorch"를 초기화.
    # wandb.init(project="titanic-pytorch")
    # wandb.init(project="titanic-pytorch", mode="offline")


    # 전처리된 데이터셋을 로드
    # get_preprocessed_dataset 함수를 호출하여 학습, 검증, 테스트 데이터셋을 가져옴.
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    # 각 데이터셋의 크기를 출력
    print("train_dataset: {0}, validation_dataset: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)))

    # DataLoader 생성
    # train_dataset과 validation_dataset을 각각 배치 크기 16으로 로드. 훈련 데이터를 섞어서 셔플링.
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    
    # test_dataset은 전체를 한 번에 로드하기 위해 batch_size를 데이터셋 크기로 설정.
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 모델 정의 및 학습
    # 입력 차원은 6 (Pclass, Sex, Age, Fare, Embarked, FamilySize), 출력 차원은 2 (생존 여부).
    my_model = MyModel_sigmoid(n_input=6, n_output=2)

    # 모델을 학습시키고, 50번의 에포크 동안 train_loader와 validation_loader를 통해 학습.
    train_model(my_model, train_data_loader, validation_data_loader, epochs=50)

    # Test 데이터셋에 대해 예측 생성
    print("[TEST DATASET]")
    test_sample = next(iter(test_data_loader))
    print("input shape: {0}".format(test_sample['input'].shape))

    output_batch = my_model(test_sample['input'])
    prediction_batch = torch.argmax(output_batch, dim=1)

    # ender_submission.csv 파일을 불러옴
    submission = pd.read_csv('gender_submission.csv')

    # 예측한 값을 'Survived' 열에 추가
    submission['Survived'] = prediction_batch.cpu().numpy()

    # 결과를 새로운 CSV 파일로 저장
    submission.to_csv('submission_sigmoid_svm.csv', index=False)

    print("예측 결과가 submission_2.csv에 저장되었습니다.")


FamilySize 열 생성 완료!
train_dataset: 712, validation_dataset: 179, test_dataset: 418
Epoch 1/50, Training Loss: 0.4153092846274376, Validation Loss: 0.3287919784585635
Epoch 2/50, Training Loss: 0.400051778058211, Validation Loss: 0.3464181733628114
Epoch 3/50, Training Loss: 0.39741280211342705, Validation Loss: 0.3229856143395106
Epoch 4/50, Training Loss: 0.394527311457528, Validation Loss: 0.31749054541190463
Epoch 5/50, Training Loss: 0.39060713946819303, Validation Loss: 0.2967113384753854
Epoch 6/50, Training Loss: 0.3857061243719525, Validation Loss: 0.31878162175416946
Epoch 7/50, Training Loss: 0.3815948837333255, Validation Loss: 0.2948843302826087
Epoch 8/50, Training Loss: 0.3795913795630137, Validation Loss: 0.3096452976266543
Epoch 9/50, Training Loss: 0.3663637323511971, Validation Loss: 0.30856217443943024
Epoch 10/50, Training Loss: 0.3538209019435777, Validation Loss: 0.3035861986378829
Epoch 11/50, Training Loss: 0.34378210968441436, Validation Loss: 0.326462478687365

# 요구사항 4

<img src="./rank.png" alt="Image Description" width="300"/>

<img src="./submission.png" alt="Image Description" width="300"/>